<h1 align="center">Momentum Strategy with Pyfolio</h1>

#

In [55]:
!pip install pyfolio
!pip install yfinance --upgrade --no-cache-dir

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.2/63.2 kB 4.3 MB/s eta 0:00:00
  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.20
    Uninstalling yfinance-0.2.20:
      Successfully uninstalled yfinance-0.2.20


In [11]:
!wget http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
!tar -xzf ta-lib-0.4.0-src.tar.gz
%cd ta-lib/
!./configure --prefix=/usr
!make
!make install
!pip install TA-Lib

--2023-06-25 07:10:39--  http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz
Resolving prdownloads.sourceforge.net (prdownloads.sourceforge.net)... 204.68.111.105
Connecting to prdownloads.sourceforge.net (prdownloads.sourceforge.net)|204.68.111.105|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2023-06-25 07:10:39--  http://downloads.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving downloads.sourceforge.net (downloads.sourceforge.net)... 204.68.111.105
Reusing existing connection to prdownloads.sourceforge.net:80.
HTTP request sent, awaiting response... 302 Found
Location: http://zenlayer.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz [following]
--2023-06-25 07:10:39--  http://zenlayer.dl.sourceforge.net/project/ta-lib/ta-lib/0.4.0/ta-lib-0.4.0-src.tar.gz
Resolving zenlayer.dl.s

In [48]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib
import pyfolio as pf
import empyrical as em
import warnings
warnings.filterwarnings("ignore")

## Step 1: Get OHLC data

In [49]:
# Define a list of stock symbols
stock_symbols = ['AAPL', 'GOOGL', 'MSFT', 'AMZN']

# Define the start and end dates for the data
start_date = '2020-01-01'
end_date = '2023-06-25'

# Fetch the OHLC data using Yahoo Finance API
def get_ohlc_data(symbol, start_date, end_date):
    data = yf.download(symbol, start=start_date, end=end_date)
    return data

# Create an empty DataFrame to store the OHLC data
ohlc_data = pd.DataFrame()

# Fetch the OHLC data for each stock symbol and append it to the DataFrame
for symbol in stock_symbols:
    data = get_ohlc_data(symbol, start_date, end_date)
    data['Symbol'] = symbol  # Add a column for the stock symbol
    ohlc_data = ohlc_data.append(data)

# Print the OHLC data
ohlc_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume,Symbol
Date,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.347931,135480400,AAPL
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.634850,146322800,AAPL
2020-01-06,73.447502,74.989998,73.187500,74.949997,73.213615,118387200,AAPL
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.869293,108872000,AAPL
2020-01-08,74.290001,76.110001,74.290001,75.797501,74.041489,132079200,AAPL
...,...,...,...,...,...,...,...
2023-06-16,127.709999,127.900002,125.300003,125.489998,125.489998,84188100,AMZN
2023-06-20,124.970001,127.250000,124.500000,125.779999,125.779999,56930100,AMZN
2023-06-21,125.639999,126.730003,123.849998,124.830002,124.830002,52137700,AMZN


## Step 2: Custom function to get top 5 performers based on 52-week rolling returns

In [50]:
def get_top_performers(data):
    # Calculate the 52-week rolling returns
    data['Returns'] = data['Close'].pct_change(52)

    # Sort the data by returns in descending order
    data = data.sort_values('Returns', ascending=False)

    # Select the top 5 performers
    top_performers = data.head(5)

    return top_performers

# Get the top 5 performers based on 52-week rolling returns
top_performers = get_top_performers(ohlc_data)

top_performers

,Open,High,Low,Close,Adj Close,Volume,Symbol,Returns
Date,,,,,,,,
2020-01-17,167.419998,167.470001,165.429993,167.100006,161.692337,34371700,MSFT,0.611224
2020-01-30,174.050003,174.050003,170.789993,172.779999,167.188538,51597500,MSFT,0.603229
2020-01-16,164.350006,166.240005,164.029999,166.169998,160.792435,23865400,MSFT,0.600096
2020-01-29,167.839996,168.750000,165.690002,168.039993,162.601913,34754500,MSFT,0.591740
2020-01-31,172.210007,172.399994,169.580002,170.229996,164.721039,36142700,MSFT,0.585748


- All top 5 stocks are from Microsoft.

## Step 3: Implement the momentum strategy

In [51]:
# Define the stop loss multiplier
stop_loss_multiplier = 2

# Define a function to calculate Average True Range (ATR)
def calculate_atr(data):
    high = data['High'].values
    low = data['Low'].values
    close = data['Close'].values
    atr = talib.ATR(high, low, close, timeperiod=14)
    return atr[-1]

# Define a function to go long on the top 5 performers with stop loss
def go_long_with_stop_loss(data, stop_loss_multiplier):
    # Calculate the entry price as the close price of the latest available data
    entry_date = data.index[-1].strftime('%Y-%m-%d')
    entry_price = data['Close'].iloc[-1]

    # Calculate the stop loss level
    atr = calculate_atr(data)
    stop_loss = entry_price - (stop_loss_multiplier * atr)

    # Print the values
    print("Symbol:", data['Symbol'].iloc[-1])
    print("Entry Date:", entry_date)
    print("Entry Price:", entry_price)
    print("Stop Loss:", stop_loss)
    print()

# Go long on the top 5 performers with stop loss
for index, row in top_performers.iterrows():
    symbol = row['Symbol']
    entry_date = index.strftime('%Y-%m-%d')
    entry_price = row['Close']
    stock_data = ohlc_data[(ohlc_data['Symbol'] == symbol) & (ohlc_data.index <= index)]
    go_long_with_stop_loss(stock_data, stop_loss_multiplier)


Symbol: MSFT
Entry Date: 2020-01-17
Entry Price: 167.10000610351562
Stop Loss: nan

Symbol: MSFT
Entry Date: 2020-01-30
Entry Price: 172.77999877929688
Stop Loss: 167.1350629306802

Symbol: MSFT
Entry Date: 2020-01-16
Entry Price: 166.1699981689453
Stop Loss: nan

Symbol: MSFT
Entry Date: 2020-01-29
Entry Price: 168.0399932861328
Stop Loss: 162.8854484900264

Symbol: MSFT
Entry Date: 2020-01-31
Entry Price: 170.22999572753906
Stop Loss: 164.53112716121754



## Step 4: Rebalance the stocks on a weekly basis

In [28]:
import time

while True:
    # Get the current date
    current_date = pd.Timestamp.now().strftime('%Y-%m-%d')

    # Check if it's a new week (e.g., Monday)
    if pd.Timestamp(current_date).dayofweek == 0:
        # Get the top 5 performers based on 52-week rolling returns
        top_performers = get_top_performers(ohlc_data)

        # Go long on the top 5 performers with stop loss
        for symbol in top_performers['Symbol']:
            stock_data = ohlc_data[(ohlc_data['Symbol'] == symbol) & (ohlc_data.index.date < current_date)]
            go_long_with_stop_loss(stock_data, stop_loss_multiplier)

    # Wait for a week (7 days) before rebalancing
    time.sleep(7 * 24 * 60 * 60)

KeyboardInterrupt: ignored

## Step 5: Perform Backtest and Analyze Strategy Using Pyfolio

In [52]:
# Create an empty portfolio to store the strategy's trades and positions
portfolio = pd.DataFrame(index=top_performers.index)
portfolio['Symbol'] = top_performers['Symbol']
portfolio['Positions'] = 0
portfolio['Entry Price'] = 0.0
portfolio['Exit Price'] = 0.0
portfolio['Returns'] = 0.0


# Iterate over the top performers and execute the strategy by going long on each stock
for index, row in top_performers.iterrows():
    symbol = row['Symbol']
    entry_date = index.strftime('%Y-%m-%d')
    entry_price = row['Close']
    stock_data = ohlc_data[(ohlc_data['Symbol'] == symbol) & (ohlc_data.index <= index)]

    # Calculate stop loss
    atr = calculate_atr(stock_data)
    stop_loss = entry_price - (stop_loss_multiplier * atr)

    # Update portfolio with trade information
    portfolio.loc[index, 'Symbol'] = symbol
    portfolio.loc[index, 'Positions'] = 1
    portfolio.loc[index, 'Entry Price'] = entry_price
    portfolio.loc[index, 'Exit Price'] = stop_loss

    # Calculate returns
    stock_returns = stock_data['Close'].pct_change()
    portfolio.loc[index, 'Returns'] = stock_returns.iloc[-1]

portfolio

,Symbol,Positions,Entry Price,Exit Price,Returns
Date,,,,,
2020-01-17,MSFT,1,167.100006,NaN,0.005597
2020-01-30,MSFT,1,172.779999,167.135063,0.028208
2020-01-16,MSFT,1,166.169998,NaN,0.018323
2020-01-29,MSFT,1,168.039993,162.885448,0.015593
2020-01-31,MSFT,1,170.229996,164.531127,-0.014759


In [53]:
# Calculate the cumulative returns of the portfolio
portfolio['Cumulative Returns'] = (portfolio['Returns'] + 1).cumprod()
portfolio

,Symbol,Positions,Entry Price,Exit Price,Returns,Cumulative Returns
Date,,,,,,
2020-01-17,MSFT,1,167.100006,NaN,0.005597,1.005597
2020-01-30,MSFT,1,172.779999,167.135063,0.028208,1.033962
2020-01-16,MSFT,1,166.169998,NaN,0.018323,1.052908
2020-01-29,MSFT,1,168.039993,162.885448,0.015593,1.069326
2020-01-31,MSFT,1,170.229996,164.531127,-0.014759,1.053544


In [54]:
# Generate the analysis report using Pyfolio
returns = portfolio['Cumulative Returns']
strategy_stats = pf.timeseries.perf_stats(returns).to_frame()
strategy_stats

,0
Annual return,1.530924e+78
Cumulative returns,3.458679e+01
Annual volatility,3.874683e-01
Sharpe ratio,6.783858e+02
Calmar ratio,NaN
Stability,9.999838e-01
Max drawdown,0.000000e+00
Omega ratio,NaN
Sortino ratio,inf
Skew,-6.373250e-01
